In [30]:
import json
import math
import gurobipy as gp
from gurobipy import Model, GRB, quicksum



In [31]:
problem = 'tests/drone_problem_21.json'
with open(problem, 'r') as file:
    data = json.load(file)

# Extract data from JSON
drone_weight = data['drone_weight']
max_capacity = data['max_capacity']
battery_capacity = data['battery_capacity']
points = data['pontos']

points


[{'x': 195, 'y': 501, 'peso': 0},
 {'x': 302, 'y': 550, 'peso': 10},
 {'x': 341, 'y': 519, 'peso': 10},
 {'x': 318, 'y': 475, 'peso': 10},
 {'x': 358, 'y': 435, 'peso': 10},
 {'x': 394, 'y': 483, 'peso': 10},
 {'x': 390, 'y': 402, 'peso': 10},
 {'x': 307, 'y': 384, 'peso': 10},
 {'x': 363, 'y': 370, 'peso': 10},
 {'x': 406, 'y': 331, 'peso': 10},
 {'x': 459, 'y': 410, 'peso': 10},
 {'x': 459, 'y': 484, 'peso': 10},
 {'x': 467, 'y': 348, 'peso': 10},
 {'x': 333, 'y': 288, 'peso': 10},
 {'x': 240, 'y': 316, 'peso': 10},
 {'x': 251, 'y': 387, 'peso': 10},
 {'x': 275, 'y': 336, 'peso': 10},
 {'x': 225, 'y': 223, 'peso': 10},
 {'x': 278, 'y': 273, 'peso': 10},
 {'x': 365, 'y': 208, 'peso': 10},
 {'x': 428, 'y': 296, 'peso': 10},
 {'x': 517, 'y': 219, 'peso': 10},
 {'x': 195, 'y': 501, 'peso': 0}]

In [32]:
n = len(points) - 2  
base_start = 0
base_end = n + 1


d = {}
for i in range(len(points)):
    for j in range(len(points)):
        if i != j:
            d[i, j] = math.sqrt((points[i]['x'] - points[j]['x'])**2 + (points[i]['y'] - points[j]['y'])**2)
        else:
            d[i, j] = 0


p = {}
for i in range(1, len(points) - 1):
    p[i] = points[i]['peso']


p[base_start] = 0
p[base_end] = 0


In [33]:
def my_callback(model, where):
    if where == GRB.Callback.MIPSOL:  # Triggered when a new solution is found
        # Get the time since the start of optimization
        solution_time = model.cbGet(GRB.Callback.RUNTIME)
        # Retrieve the objective value of the solution
        obj_val = model.cbGet(GRB.Callback.MIPSOL_OBJ)
        
        # Store the results
        results.append({'time': solution_time, 'objective_value': obj_val})

env = gp.Env(empty=True)
env.setParam('LICENSEID', 2549422)
env.start()
model=Model('drone')
model.setParam('TimeLimit', 7200)
alpha = 1
beta = 1
x = model.addVars(len(points), len(points), vtype=GRB.BINARY, name="x")
omega = model.addVars(len(points), vtype=GRB.CONTINUOUS, name="omega")
e = model.addVars(len(points), len(points), vtype=GRB.CONTINUOUS, name="e")


#1. cada casa so vai para 1 lugar
for i in range(1, n + 1):
    model.addConstr(quicksum(x[i, j] for j in range(1, n + 2)) == 1, name=f"c1_{i}")
#2.nao vai da base para a base
model.addConstr(x[base_start, base_end] == 0, name="c2")
#3. apos voltar, nao sai
for j in range(len(points)):
    model.addConstr(x[base_end, j] == 0, name=f"c3_{j}")
#4. nao vai da casa para ela mesma
for i in range(len(points)):
    model.addConstr(x[i, i] == 0, name=f"c4_{i}")
#5. nao vai para base inicial
for i in range(len(points)):
    model.addConstr(x[i, base_start] == 0, name=f"c5_{i}")
#6. entra na base o mesmo numero de vezes que sai
model.addConstr(quicksum(x[i, base_end] for i in range(len(points))) == quicksum(x[base_start, j] for j in range(len(points))), name="c6")
#7. cada casa eh visitada 1 vez
for j in range(1, n + 1):
    model.addConstr(quicksum(x[i,j] for i in range(len(points))) == 1, name=f"c7_{j}")
#8. nenhum omega é maior que a capacidade
for i in range(len(points)):
    model.addConstr(omega[i] <= max_capacity, name=f"c8_{i}")
#9. omega da base final é o peso do drone
model.addConstr(omega[base_end] == drone_weight, name="c9")
# 10. Calculo de omega
for i in range(1, n+1):
    model.addConstr(omega[i] == p[i] + quicksum((omega[j])*x[i,j] for j in range(len(points))), name=f"c10_{i}")
# 11. Calculo de e inicial
for j in range(len(points)):
    model.addConstr(e[base_start, j] == battery_capacity - (((d[base_start, j])) * ((omega[j]) * x[base_start, j])), name=f"c11_{j}")
#12. Calculo de e
for i in range(1, n+2):
    for j in range(len(points)):
        model.addConstr(e[i, j] == quicksum(e[k,i]*x[k,i] for k in range(len(points))) - (((d[i, j])) * ((omega[j]) * x[i, j])), name=f"c12_{i}_{j}")
#13. e deve ser maior ou igual a 0
for i in range(len(points)):
    for j in range(len(points)):
        model.addConstr(e[i, j] >= 0, name=f"c13_{i}_{j}")

model.setObjective(quicksum(x[i,j] * (d[i,j]) * omega[j] for i in range(len(points)) for j in range(len(points))), GRB.MINIMIZE)

results = []

model.optimize(my_callback)










Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2549422
Academic license 2549422 - for non-commercial use only - registered to gu___@gmail.com
Set parameter TimeLimit to value 30
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 2549422 - for non-commercial use only - registered to gu___@gmail.com
Optimize a model with 666 rows, 1081 columns and 1612 nonzeros
Model fingerprint: 0xb30d5e8b
Model has 504 quadratic objective terms
Model has 550 quadratic constraints
Variable types: 552 continuous, 529 integer (529 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 4e+02]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [8e+01, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
  QRHS range       [1e+01, 1e

In [34]:
for var in model.getVars():
    if(var.x > 0):
        print(f"{var.varName} = {var.x}")


#print solution objective value
print(f"Objetivo: {model.objVal}")

x[0,1] = 1.0
x[0,4] = 1.0
x[0,5] = 1.0
x[0,9] = 1.0
x[0,14] = 1.0
x[0,15] = 1.0
x[0,16] = 1.0
x[1,2] = 1.0
x[2,3] = 1.0
x[3,22] = 1.0
x[4,6] = 1.0
x[5,11] = 1.0
x[6,12] = 1.0
x[7,8] = 1.0
x[8,22] = 1.0
x[9,20] = 1.0
x[10,22] = 1.0
x[11,10] = 1.0
x[12,22] = 1.0
x[13,19] = 1.0
x[14,18] = 1.0
x[15,7] = 1.0
x[16,13] = 1.0
x[17,22] = 1.0
x[18,17] = 1.0
x[19,22] = 1.0
x[20,21] = 1.0
x[21,22] = 1.0
omega[1] = 40.0
omega[2] = 30.0
omega[3] = 20.0
omega[4] = 40.0
omega[5] = 40.0
omega[6] = 30.0
omega[7] = 30.0
omega[8] = 20.0
omega[9] = 40.0
omega[10] = 20.0
omega[11] = 30.0
omega[12] = 20.0
omega[13] = 30.0
omega[14] = 40.0
omega[15] = 40.0
omega[16] = 40.0
omega[17] = 20.0
omega[18] = 30.0
omega[19] = 20.0
omega[20] = 30.0
omega[21] = 20.0
omega[22] = 10.0
e[0,0] = 140000000.0
e[0,1] = 139995292.5590816
e[0,2] = 140000000.0
e[0,3] = 140000000.0
e[0,4] = 139992965.797842
e[0,5] = 139992007.50351894
e[0,6] = 140000000.0
e[0,7] = 140000000.0
e[0,8] = 140000000.0
e[0,9] = 139989161.47611526
e[0,1

In [35]:
subroutes = []
goesTo = {}

for var in model.getVars():
    if var.VarName[0] == 'x' and var.x > 0:
        #print(f"{var.VarName} = {var.x}")
        x,y = var.VarName.split('[')[1].split(']')[0].split(',')
        x = int(x)
        y = int(y)
        if x == base_start:
            subroutes.append([y])
        else:
            goesTo[x] = y

for i in range(len(subroutes)):
    next = goesTo[subroutes[i][-1]]
    while next != base_end:
        subroutes[i].append(next)
        next = goesTo[next]

print(subroutes)
route = []
for subroute in subroutes:
    route += subroute
    route.append(0)
route.pop()
print(route)
print(model.ObjVal)
        

[[1, 2, 3], [4, 6, 12], [5, 11, 10], [9, 20, 21], [14, 18, 17], [15, 7, 8], [16, 13, 19]]
[1, 2, 3, 0, 4, 6, 12, 0, 5, 11, 10, 0, 9, 20, 21, 0, 14, 18, 17, 0, 15, 7, 8, 0, 16, 13, 19]
93137.62595279103


In [36]:
solution_count = model.SolCount
solve_json = {}
problem_name = problem.split('/')[1].split('.')[0]
with open(f"results/solved_{problem_name}.json", 'w') as file:
    json.dump(results, file)